In [31]:
import json
import requests

import dlt
from dlt.destinations import qdrant

from qdrant_client import QdrantClient
client = QdrantClient("http://localhost:6333")

In [2]:
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [5]:
print(f'dlt version: {dlt.__version__}')

dlt version: 1.12.3


In [ ]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

2025-07-13 19:34:59,047|[WARNING]|17167|131809651732608|dlt|pipeline.py|_state_to_props:1680|The destination dlt.destinations.duckdb:None in state differs from destination dlt.destinations.qdrant:qdrant in pipeline and will be ignored


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Run started at 2025-07-14 02:34:59.054410+00:00 and COMPLETED in 44.88 seconds with 4 steps.
Step extract COMPLETED in 1.41 seconds.

Load package 1752460533.1248963 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.07 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752460533.1248963 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 9.33 seconds.
Pipeline zoomcamp_pipeline load step completed in 9.32 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/rajinder-mavi/code/nlp_sandbox/llm_zoomcamp/db.qdrant location to store data
Load package 1752460533.1248963 is LOADED and contains no failed jobs

Step run COMPLETED in 44.88 seconds.
Pipeline zoomcamp_pipeline load step completed in 9.32 seconds
1 load package(s) were l

In [34]:
with open('db.qdrant/meta.json', 'r') as file:
    dlt_meta = json.load(file)

In [40]:
dlt_meta['collections']['zoomcamp_tagged_data']['vectors']

{'fast-bge-small-en': {'size': 384,
  'distance': 'Cosine',
  'hnsw_config': None,
  'quantization_config': None,
  'on_disk': None,
  'datatype': None,
  'multivector_config': None}}